Team members

*   cs23mtech11030 - Parasa Naga Venkata Pavan
*   cs23mtech11021 - Abhishree Gajanan Khangar
*   cs23mtech11032 - Pulicharla Punith Kumar
*   cs23mtech11004 - Apurba Saha






In [1]:
import numpy as np
from sympy import *
import random

In [2]:
#Importing data from csv file
import csv
data2=[]
file_path = 'input_bounded.csv'
with open(file_path, 'r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)

    # Iterate through each row in the CSV file
    for row in csv_reader:
        # Each row is a list of values
        data2.append(row)

In [3]:
#Converting char data to int
data=[]
for i in data2:
    l=[]
    for j in i:
        l.append(float(j))
    data.append(l)



In [4]:
print("Data",data)

Data [[5.0, 4.0, 0.0], [5.0, 2.0, 0.0], [-2.0, 1.0, 5.0], [-3.0, -1.0, -3.0], [2.0, -3.0, 12.0], [-1.0, 0.0, 0.0], [0.0, -1.0, 0.0], [1.0, 1.0, 10.0]]


In [5]:
z=data[0][0:-1]
c=data[1][0:-1]
data1=data[2:]


In [6]:
a=[]
b=[]
for i in data1:
    a.append(i[:-1])
    b.append(i[-1])

m=len(a)
n=len(a[0])
a=np.array(a)
b=np.array(b)
z=np.array(z)


In [7]:
#Function to check if point is in region
def point_in_region(a,b,x):
    return a @ x <= b

In [8]:
#Step 1: Checking given point is in region
def step1(a,b,z):
    check=point_in_region(a,b,z)
    for i in check:
        if not i:
            return False
    return True

In [9]:
#Function to no. of tight and untight rows
def no_of_tight(a,b,x):
    tight_count=0
    for i in range(len(a)):
        if round(a[i] @ x.T,4) ==b[i]:
            tight_count+=1
    return tight_count

In [10]:
#Function to check if point is vertex
def check_vertex(a1):
    if len(a1)==0:
        #print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        return False
    if np.linalg.matrix_rank(a1)==n:
        return True
    else:
        return False

In [11]:
#Function to get tight and untight rows
def spliting_matrix(a,b,x):
    a1=[]
    a2=[]
    b1=[]
    b2=[]
    #print(a)
    for i in range(a.shape[0]):
        # print("::::::::::::::::::")
        # print(a[i])
        # print(x.T)
        # print(b[i])
        # print(a[i] @ x.T)
        # print("!!!!!!!!!!!!!!!!!!")
        if round(a[i] @ x.T,4) ==round(b[i],4):
            a1.append(a[i])
            b1.append(b[i])
        else:
            a2.append(a[i])
            b2.append(b[i])
    a1=np.array(a1)
    a2=np.array(a2)
    b1=np.array(b1)
    b2=np.array(b2)
    return a1,a2,b1,b2

In [12]:
def replace_zero(ar):
  eps = 0.0000000000000001
  count = 0
  for i in ar:
    if(i == 0):
      i = eps
      ar[count] = i
    count+=1
  return ar

In [13]:
#Step 2: Function to get vertex
def step2(a,b,x):
    tight_rows=no_of_tight(a,b,x)
    a1,a2,b1,b2=spliting_matrix(a,b,x)
    while not check_vertex(a1):
        # print("================")
        #x=new_point(a1,a2,b1,b2,x)
        if len(a1)==0:
            a1=np.array([]).reshape((0, n))
        u = np.array(Matrix(a1).nullspace()).astype(float)
        #print(a1)
        # print(Matrix(a1))
        # print(u)
        u = u.flatten()
        u = np.reshape(u, (-1, n))

        u_hat = u / np.linalg.norm(u,axis=1,keepdims=True)
        u_hat=u_hat[0]

        di_alpha = (np.dot(a2 , u_hat))

        di_alpha = replace_zero(di_alpha)

        alpha  = (b2 -np.dot(a2,x.T) )/ di_alpha.T
        #print("pre alpha",alpha)
        #alpha = alpha[0]
        # print("aplha",alpha)
        alpha_1 = []
        # removing -ve alphas
        for aa in alpha:
            if aa > 0:
                alpha_1.append(aa)
        # taking min of alphas

        alpha_1 = min(alpha_1)
        # alpha_1 = min(alpha_1)
        u_hat = u_hat.T
        # print("alpha r",alpha_1)
        # print("before x",x)
        x = x + alpha_1 * u_hat

        print("x",x)
        a1,a2,b1,b2=spliting_matrix(a,b,x)
    return x

In [14]:
step2(a,b,z)

x [6. 4.]
x [1.66666667 8.33333333]


array([1.66666667, 8.33333333])

In [15]:
#Function to check given point is optimum
def check_optimum(a1,c):
    #print("^^^^^^^^^^^^^^^^^^")
    #print(a1)
    alphas=np.linalg.solve(a1.T, c)
    #print(alphas)
    for i in alphas:
        if i<0:
            return False
    return True

In [16]:
#Function to get optimum point
def step3(a,b,x,c):
    a1,a2,b1,b2=spliting_matrix(a,b,x)
    while not check_optimum(a1,c):
        # print("****************************************")
        print("Present Vertex point",x)
        a1i=np.linalg.inv(a1).T
        alphas=np.linalg.solve(a1.T, c)
        # print("alphas:",alphas)
        # print(len(alphas))
        u=[]
        for i in range(len(alphas)):
            if alphas[i] < 0:
                u=a1i[i]
                break
        u=np.array(u)
        u=-1*u
        u=u.T
        di_alpha = np.dot(a2 , u)

        di_alpha = replace_zero(di_alpha)

        alpha  = (b2 -np.dot(a2,x.T) )/ di_alpha
        # print("--------------")
        # print("beta:",alpha)
        #alpha = alpha[0]
        #print(alpha)
        alpha_1 = []
        # removing -ve alphas
        for aa in alpha:
            if aa > 0:
                alpha_1.append(aa)
        # taking min of alphas
        alpha_1 = min(alpha_1)
        u_hat = u.T
        #print("before x:",x)
        x = x + alpha_1 * u_hat

        # print("x=",x)
        a1,a2,b1,b2=spliting_matrix(a,b,x)
    return x

In [17]:
if step1(a,b,z):
  res=step3(a,b,step2(a,b,z),c)
  print("Optimum Point",res)
else:
  print("Given point is not optimum")

x [6. 4.]
x [1.66666667 8.33333333]
Present Vertex point [1.66666667 8.33333333]
Optimum Point [8.4 1.6]
